# Climate Risk Analysis: Mongolia Dzud and Summer Drought Analysis 2009-2010

## Introduction

Mongolia is highly vulnerable to climate extremes due to its cold continental climate and strong dependence on pastoral livestock subsistence farming. One of the most severe climate hazards affecting the country is *dzud*, a compound winter disaster of varying types, usually characterised by extreme cold, long term or deep snow cover, and ice crusts that prevent livestock from accessing pasture and damage or make infrastructure unusable. Dzud can lead to major livestock losses, threatening rural subsistence farmers and national level food security, further increasing urban struggles with overcrowding and over worked services.

**Types of dzud**
- **White Dzud**: Deep snow covers the ground, burying grass and infrastructure.
- **Black Dzud**: Little persistent snow combined with poor rainfall in the summer, causing little grass on the pastures.
- **Iron dzud**: Icy impenetrable crust forms over the ground, making grass impossible to access.

The 2009-2010 dzud stands out as one of the worst climate disasters in Mongolia's modern history, resulting in the loss of many millions of liverstock and serious socio economic impacts across the entire country. Importantly, dzud risk deos not arise from the winter alone. A summer of poor pasture growth, typically caused by below average rainfall or drought, can leave livestock in a more precarious position going into winter amplifying the potential extremes of the winter.

This notebook analyses the 2009-2010 dzud as a *compound climate event*, starting in the summer 2009 and its precipitation conditions and finishing in the extreme winter spanning 2009-2010. By analysing both phases together, this analysis aims to show the full climatic pathway through which the risk appeared in Mongolia during this period.

**Objectives**
- Assess the precipitation levels and spread in the 2009 summer as a preconditioning factor in dzud risk using satellite rainfall data.  
- Demonstrate the severity of the winter conditions in 2009 and 2010 using temperature and snow related indicators.  
- Compare how the summer and winter of 2009-2010 conditions are against the historical baseline to determine their extremes.    
- Examine spatial patterns of climate stress in Mongolia and illustrate local conditions using Ulaanbaatar as a representative location.  
- Provide a seasonal and compound perspective on climate risk that later supports a comparison with the drought and extreme rainfall risk in Chad.

## Data Sources

This analysis combines both satellite data and reanalysis data to capture the different climatic drivers of dzud risk across seasons and years. Summer conditions, especially those of the pastures and rural areas, are going to be analysed using precipitation data, while winter severity for dzud using temperature and snow variables.

### Primary Sources

**ERA5 / ERA5‑Land Reanalysis (ECMWF)**    
- *Variables*: 
    - 2m air temperature  
    - Snow depth (and/or cover)  
- *Spatial resolution*:  
    - ERA5: ~0.25° x 0.25°   
    - ERA5-Land: ~0.10° x 0.10°  
- *Temporal resolution*:
    - Hourly, being aggregated to daily for this analysis
- *Coverage*:
    - Global
- *Usage in this notebook*:
    - Assessment of the winter severity during 2009-10 dzud
    - Historical baseline comparison of winter conditions
- *Source*:
    - https://cds.climate.copernicus.eu/datasets/

**CHIRPS (Climate Hazards Group InfraRed precipitation with Station Data)**
- *Variables*: 
    - Daily Precipitation
- *Spatial resolution*:  
    - ~0.25° x 0.25°     
- *Temporal resolution*:
    - Daily
- *Coverage*:
    - Global, 1981-Current
- *Usage in this notebook*:
    - Assessment of the summer precipitation conditions (June-October 2009)
    - Historical baseline comparison of summer rainfall
- *Source*:
    - https://data.chc.ucsb.edu/products/CHIRPS-2.0/

### Supplementary Sources

**Country Boundaries**:
- Source: Natural Earth (110m cultural vectors)
- URL: https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip
- Purpose: Spatial clipping and visualization

**MODIS Snow Cover (Potentially)**  
- *Variable*: 
    - Snow cover extent  
- *Spatial resolution*: 
    - ~500 m  
- *Temporal resolution*: 
    - Daily  
- *Usage*:  
    - Qualitative validation of snow persistence patterns  
    - Supplementary visualisation of winter snow conditions  
- *Source*:
    - https://modis.gsfc.nasa.gov/data/dataprod/mod10.php

### Data Notes

- The analysis period spans **June 2009 to March 2010**, capturing both summer preconditioning and winter dzud severity.  
- Historical baseline conditions are derived from **1981–2009**, providing a consistent reference period prior to the 2009–2010 event.    
- All datasets are spatially subset to Mongolia’s national boundaries.  
- Temperature is analysed in degrees Celsius and precipitation in millimeters.  
- Seasonal definitions follow climatological relevance rather than calendar years:
  - **Summer**: June–October
  - **Winter**: November–March
- ERA5 data is installed using cdsapi and an API key. This requires prior configuration of the cdsapi and a valid CDS account.
- MODIS may be used to add further clarity and visualisation of specifically 2009-2010 winter, however due to the nature of the dataset, it may be out of the scope of this notebook

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import rioxarray
import matplotlib.colors as colors
import cdsapi
import os

c = cdsapi.Client()

out_dir = "C:\Projects\geodataprojects\data\mongolia\era5"
os.makedirs(out_dir, exist_ok=True)

years = range(1981, 2011)
all_exists = True

for year in years:
    outfile = f"{out_dir}/era5_mongolia_winter_{year}.nc"

    if os.path.exists(outfile):
        continue
    else:
        all_exists = False
        print(f"Downloading ERA5 winter data for {year}...")

    c.retrieve(
        "reanalysis-era5-land",
        {
            "variable": [
                "2m_temperature",
                "snow_depth",
            ],
            "year": year,
            "month": ["11", "12", "01", "02", "03"],
            "day": [f"{d:02d}" for d in range(1, 32)],
            "time": "00:00",
            "area": [
                52, 87,   
                41, 120,
            ],
            "format": "netcdf",
        },
        outfile,
    )

if all_exists == True:
    print("All files are already installed")
    

2025-12-30 00:52:13,891 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.


All files are already installed


## Historical Baseline Context

### Why 1981-2009?

### Statistical References

## Summer 2009 Preconditioning Analysis

## Winter 2009-2010 Dzud Severity Analysis

## Localised Summer and Winter Analysis: Ulaanbaatar

## Seasonal Patterns and Compounding Risks

## Summary Statistics

## Conclusion and Next Steps